In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import partitura as pt
import partitura.musicanalysis as musan
from ic.data import DataPiece, Piece
import importlib
from CIA.getters import get_dataloader_generator
from CIA.data_processors.data_processor import DataProcessor
from CIA.getters import get_data_processor
from pathlib import Path
from pretty_midi import PrettyMIDI
import json
import re
from ic.beam_search.io import ICRes
import torch
import os
import pretty_midi
from CIA.dataloaders.piano_dataloader import PianoDataloaderGenerator
from scipy.stats import pearsonr
from ic.eval_ import get_batik_matched_note_array
import pandas as pd
import tempfile


In [2]:
batik_dir = '/share/hel/home/mathias/datasets/batik_plays_mozart'
midi_file = batik_dir+'/midi/kv331_1.mid'
perf1 = pt.load_performance_midi(midi_file)
match_file = batik_dir + '/match/kv331_1.match'
perf2, align = pt.load_match(match_file)

note_array = perf2.note_array()
n_notes = note_array.shape[0]
hehe_file = pretty_midi.PrettyMIDI(midi_file)
pitches = [n.pitch for n in sorted(hehe_file.instruments[0].notes, key=lambda x: x.start)]
# tt = musan.estimate_tonaltension(perf2)

/share/hel/home/mathias/devel/python3/partitura/partitura/io/importmidi.py:245: UserWarning: ignoring MIDI message note_off channel=1 note=64 velocity=0 time=64
  warnings.warn(f"ignoring MIDI message {msg}")


In [3]:
min_notes_in_context = 256
n_inpaint = 506
hop_size = n_inpaint-min_notes_in_context
hop_size, n_notes//hop_size
decoding_start = 514



In [4]:
n_notes//(n_inpaint-min_notes_in_context)

24

In [5]:
files = sorted(Path(batik_dir, 'midi').glob('*.mid'))
pieces = []
for f in files:
    n_notes = len(PrettyMIDI(str(f)).instruments[0].notes)
    for i in range(-min_notes_in_context, n_notes-n_inpaint, hop_size):
        pieces.append(dict(
            path=str(f),
            start_node=i,
            n_inpaint=n_inpaint,
            end_window=0,
        # ) for i in range(0-min_notes_in_context-1, n_notes, hop_size)],
        ))
pieces

with open('configs/batik_all_pieces.json', 'w') as f:
    json.dump(pieces, f, indent=2)

In [6]:
dp = DataPiece(
    label='batik',
    pieces=[Piece(
        midi_file,
        start_node=i,
        n_inpaint=n_inpaint,
        end_window=0,
    # ) for i in range(0-min_notes_in_context-1, n_notes, hop_size)],
    ) for i in range(-min_notes_in_context, n_notes, hop_size)],
    cache_path='/share/hel/home/mathias/.cache/mutdata/pia',
    # pad_before=True,
)
model_dir = 'models/piano_event_performer_2021-10-01_16:03:06'
config = importlib.import_module('.config_autoreg', f'{model_dir.replace("/", ".")}').config
for t in ['time_dilation', 'velocity_shift', 'transposition']:
    config['dataloader_generator_kwargs']['transformations'][t] = False
# NOTE: unconmment, since we indeed want to do only inpainting
# config['data_processor_kwargs']['num_events_before'] = 1
# config['data_processor_kwargs']['num_events_after'] = 0
# TODO: check correct number...
# num_events_middle = 1024 - 6
# num_events_middle = 512 - 5 # magic - 5 for special tokens
# num_events_middle = 256
num_max_generated_events = 1024 - 6

load = True
overfitted = True

# === Decoder ====
# dataloader generator
dg : PianoDataloaderGenerator =  get_dataloader_generator(
    dataset=config["dataset"],
    dataloader_generator_kwargs=config["dataloader_generator_kwargs"],
)
dpr : DataProcessor = get_data_processor(
        dataloader_generator=dg,
        data_processor_type=config["data_processor_type"],
        data_processor_kwargs=config["data_processor_kwargs"],
    )
dp.dataloader_generator = dg

Loading index dictionnary


In [7]:
wins = list(iter(dp))
torch.cuda.is_available = lambda : False
os.environ["CUDA_VISIBLE_DEVICES"] = ''
x, metadata_dict = zip(*[dpr.preprocess(window[0], num_events_middle=n_inpaint) for window in wins])


/share/hel/home/mathias/devel/python3/PIA.No3/ic/data.py:167: UserWarning: hard coded num_before tokens
  warn('hard coded num_before tokens')
/share/hel/home/mathias/devel/python3/PIA.No3/ic/data.py:176: UserWarning: This put's the first token of 
  warn('This put\'s the first token of ')
/share/hel/home/mathias/devel/python3/PIA.No3/ic/data.py:171: UserWarning: Test that the following line works by also when piece.start <=0
  warn('Test that the following line works by also when piece.start <=0')


In [8]:
x0 = x[0][0,decoding_start:decoding_start+n_inpaint,0]
x1 = x[1][0,decoding_start+n_inpaint-hop_size:decoding_start+n_inpaint,0]
x2 = x[2][0,decoding_start+n_inpaint-hop_size:decoding_start+n_inpaint,0]

In [9]:
cated = [x[0][decoding_start:decoding_start+n_inpaint]] + [x_[decoding_start+n_inpaint-hop_size:decoding_start+n_inpaint,0] for x_ in x[1:]]
res = torch.cat(cated)

RuntimeError: Tensors must have same number of dimensions: got 3 and 2

In [ ]:
(res[:len(pitches)] == torch.tensor([dg.dataset.value2index['pitch'][i] for i in pitches])).all()

tensor(True)

# Stich the ics together

In [1]:
from ic.eval_ import get_all_token_ics_batik
piece_dict =  get_all_token_ics_batik()

In [4]:
piece_dict

{'kv333_2': {'paths': (PosixPath('out/a6aa5f7b2689124d92900e130182eebe0394953f773fcfec9b9e9e1466e792da/samplingconfig-k-traces-128-temperature-1-0-n-poly-notes-none-dynamic-temperature-max-ic-50-0-top-p-0-0-top-k-0/kv333_2_start_-256_nodes_506_end_0'),
   PosixPath('out/a6aa5f7b2689124d92900e130182eebe0394953f773fcfec9b9e9e1466e792da/samplingconfig-k-traces-128-temperature-1-0-n-poly-notes-none-dynamic-temperature-max-ic-50-0-top-p-0-0-top-k-0/kv333_2_start_-6_nodes_506_end_0'),
   PosixPath('out/a6aa5f7b2689124d92900e130182eebe0394953f773fcfec9b9e9e1466e792da/samplingconfig-k-traces-128-temperature-1-0-n-poly-notes-none-dynamic-temperature-max-ic-50-0-top-p-0-0-top-k-0/kv333_2_start_244_nodes_506_end_0'),
   PosixPath('out/a6aa5f7b2689124d92900e130182eebe0394953f773fcfec9b9e9e1466e792da/samplingconfig-k-traces-128-temperature-1-0-n-poly-notes-none-dynamic-temperature-max-ic-50-0-top-p-0-0-top-k-0/kv333_2_start_494_nodes_506_end_0'),
   PosixPath('out/a6aa5f7b2689124d92900e130182eebe03

In [81]:
(piece_dict['kv331_1']['toks'][:,0] == torch.tensor([dg.dataset.value2index['pitch'][i] for i in pitches])).all()

tensor(True)

In [ ]:
snote_array = score.note_array()
# note array of the performance
pnote_array = performance.note_array()
# indices of the notes that have been matched
matched_note_idxs = pt.musicanalysis.performance_codec.get_matched_notes(
    spart_note_array=snote_array, 
    ppart_note_array=pnote_array, 
    alignment=alignment,
)

In [5]:
from functools import partial
import pandas as pd
from ic.curves import Interpolator
from jsonargparse import ActionConfigFile, ArgumentParser

# parser = ArgumentParser(parser_mode="jsonnet")
# parser.add_argument('--weight', type=dictWeight)
# parser.add_argument('--config', action=ActionConfigFile)
# jsonnet = '''
# {local channel_weight_mod = import './configs/weights.jsonnet',
# channel_weight_mod.hann(2.0, 'normalized_weights', ['pitch', 'time_shift'])}
# }
# '''
# args = parser.parse_args(['--config', jsonnet])
# PITCHES = {
# 	0: ["C", "B#", "D--"],
# 	1: ["C#", "B##", "D-"],
# 	2: ["D", "C##", "E--"],
# 	3: ["D#", "E-", "F--"],
# 	4: ["E", "D##", "F-"],
# 	5: ["F", "E#", "G--"],
# 	6: ["F#", "E##", "G-"],
# 	7: ["G", "F##", "A--"],
# 	8: ["G#", "A-"],
# 	9: ["A", "G##", "B--"],
# 	10: ["A#", "B-", "C--"],
# 	11: ["B", "A##", "C-"],
# }

# accepted_pitches = [ii for i in PITCHES.values() for ii in i]
# pitch_to_ix = {p: accepted_pitches.index(p) for p in accepted_pitches}

# def tokenize_pitch_spelling(ps_note):
# 	# step = {"A": 0, "B": 1, "C": 2, "D": 3, "E": 4, "F": 5, "G": 6}[]
# 	alter = {0:"", 1:"#", 2:"##", -1:"-", -2:"--"}[ps_note["alter"].item()]
# 	return pitch_to_ix[ps_note["step"].item()+alter]

# def extract_features(perf_note):
# 	features = np.zeros((14,))
# 	# One hot of Pitch Class for first 12 entries
# 	features[int(perf_note["pitch"].item()%12)] = 1
# 	# pitch as float
# 	features[12] = perf_note["pitch"].item()/127
# 	# duration normalized per minute
# 	features[13] = perf_note["duration_sec"].item() / 60
# 	return features
# res = {}
from tqdm.contrib.concurrent import process_map,thread_map
import tqdm.notebook as tqdm
from ic.eval_ import compute_windowed_metric
[partial(compute_windowed_metric,piece_dict )(x) for x in  tqdm.tqdm(piece_dict.keys())]
# res = thread_map(compute_windowed_func, piece_dict.keys(), max_workers=10)
# interpolator = Interpolator(
#     metric_times = [piece_dict['kv331_1']['tok_times']],
#     metric = [piece_dict['kv331_1']['tok_ics']],
#     weight_fn = Hann(
#         window_size=2.0,
#         channel_weight=[0.9099181, 0, 0, 0.68306011]
#     )
# )

  0%|          | 0/36 [00:00<?, ?it/s]

Traceback (most recent call last):
  File "_pydevd_bundle/pydevd_cython.pyx", line 577, in _pydevd_bundle.pydevd_cython.PyDBFrame._handle_exception
  File "_pydevd_bundle/pydevd_cython.pyx", line 312, in _pydevd_bundle.pydevd_cython.PyDBFrame.do_wait_suspend
  File "/share/hel/home/mathias/miniconda3/envs/pia2/lib/python3.8/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 2070, in do_wait_suspend
    keep_suspended = self._do_wait_suspend(thread, frame, event, arg, suspend_type, from_this_thread, frames_tracker)
  File "/share/hel/home/mathias/miniconda3/envs/pia2/lib/python3.8/site-packages/debugpy/_vendored/pydevd/pydevd.py", line 2106, in _do_wait_suspend
    time.sleep(0.01)
KeyboardInterrupt


TypeError: compute_windowed_metric() missing 1 required positional argument: 'piece_dict'

In [11]:
piece_dict

{'kv333_2': {'paths': (PosixPath('out/a6aa5f7b2689124d92900e130182eebe0394953f773fcfec9b9e9e1466e792da/samplingconfig-k-traces-128-temperature-1-0-n-poly-notes-none-dynamic-temperature-max-ic-50-0-top-p-0-0-top-k-0/kv333_2_start_-256_nodes_506_end_0'),
   PosixPath('out/a6aa5f7b2689124d92900e130182eebe0394953f773fcfec9b9e9e1466e792da/samplingconfig-k-traces-128-temperature-1-0-n-poly-notes-none-dynamic-temperature-max-ic-50-0-top-p-0-0-top-k-0/kv333_2_start_-6_nodes_506_end_0'),
   PosixPath('out/a6aa5f7b2689124d92900e130182eebe0394953f773fcfec9b9e9e1466e792da/samplingconfig-k-traces-128-temperature-1-0-n-poly-notes-none-dynamic-temperature-max-ic-50-0-top-p-0-0-top-k-0/kv333_2_start_244_nodes_506_end_0'),
   PosixPath('out/a6aa5f7b2689124d92900e130182eebe0394953f773fcfec9b9e9e1466e792da/samplingconfig-k-traces-128-temperature-1-0-n-poly-notes-none-dynamic-temperature-max-ic-50-0-top-p-0-0-top-k-0/kv333_2_start_494_nodes_506_end_0'),
   PosixPath('out/a6aa5f7b2689124d92900e130182eebe03

In [ ]:
hehe = zip(*res)

In [148]:
pearsonr

<function scipy.stats.stats.pearsonr(x, y)>

In [ ]:
tt

In [134]:
tt.dtype

dtype([('onset_sec', '<f4'), ('cloud_diameter', '<f4'), ('cloud_momentum', '<f4'), ('tensile_strain', '<f4')])

In [144]:
tt[10, 'onset_sec']

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [136]:
tt[:10, ['onset_sec', 'cloud_diameter']]

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

torch.Size([1997, 4])

In [123]:
len(performance.note_array())

1997

In [115]:
interpolator([torch.arange(0,10, 0.1)])[0]

tensor([[ 3.3390,  0.0000,  0.0000,  0.0000],
        [ 6.7259,  0.0000,  0.0000,  2.0885],
        [ 6.6187,  0.0000,  0.0000,  2.0603],
        [ 6.4316,  0.0000,  0.0000,  2.0071],
        [ 6.1694,  0.0000,  0.0000,  1.9302],
        [ 5.8384,  0.0000,  0.0000,  1.8316],
        [ 5.4467,  0.0000,  0.0000,  1.7137],
        [ 5.0041,  0.0000,  0.0000,  1.5793],
        [ 7.1186,  0.0000,  0.0000,  6.7323],
        [ 6.5829,  0.0000,  0.0000,  6.5185],
        [ 6.0002,  0.0000,  0.0000,  6.2352],
        [ 7.3348,  0.0000,  0.0000,  9.3339],
        [ 6.6730,  0.0000,  0.0000,  8.8817],
        [ 5.9862,  0.0000,  0.0000,  8.3445],
        [ 5.2912,  0.0000,  0.0000,  7.7355],
        [ 4.6052,  0.0000,  0.0000,  7.0696],
        [11.0506,  0.0000,  0.0000, 10.6391],
        [10.3600,  0.0000,  0.0000,  9.8720],
        [ 9.6412,  0.0000,  0.0000,  9.0481],
        [ 8.9117,  0.0000,  0.0000,  8.1877],
        [ 8.1896,  0.0000,  0.0000,  7.3121],
        [ 7.4645,  0.0000,  0.0000